In [180]:
import pandas as pd
import numpy as np
from datetime import time
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [378]:

def SOURCE_DATA_MANIPULATE(TAKE_PROFIT, ROOT, DATE, k):

    
    FILES_1 = ['XRPUSDT - 4 HOUR - 0.5% - NON SAFE - NON SL',
               'ETHUSDT - 4 HOUR - 0.3% - NON SAFE - NON SL',
               'ADAUSDT - 4 HOUR - 0.3% - NON SAFE - NON SL',
               'BTCUSDT - 1 HOUR - 0.3% - SAFE - NON SL',
               'AVAXUSDT - 4 HOUR - 0.3% - NON SAFE - NON SL']

    FILES_2 = ['INDICATORS - 1 DAY PER 1 HOUR - XRPUSDT',
               'INDICATORS - 1 DAY PER 1 HOUR - ETHUSDT',
               'INDICATORS - 1 DAY PER 1 HOUR - ADAUSDT',
               'INDICATORS - 1 DAY PER 1 HOUR - BTCUSDT',
               'INDICATORS - 1 DAY PER 15 MIN - AVAXUSDT']

    DF_TRADES      = pd.read_csv(ROOT + '/' + FILES_1[k] + '.csv')
    DF_DATA        = pd.read_csv(ROOT + '/' + FILES_2[k] + '.csv')
    DATE_MAX       = '2023-03-18'


    DF_TRADES      = DF_TRADES.iloc[::-1]
    DF_TRADES      = DF_TRADES.iloc[::2].reset_index()

    del DF_DATA['Unnamed: 0']

    DF_TRADES['Date/Time']       = pd.to_datetime(DF_TRADES['Date/Time'])
    DF_DATA['datetime']          = pd.to_datetime(DF_DATA['datetime'])
    DF_DATA['datetime']          = DF_DATA['datetime'] + pd.DateOffset(hours=1)

    DF_DATA = DF_DATA[DF_DATA['datetime'] >= DATE].reset_index().iloc[:, 1:]
    DF_TRADES = DF_TRADES[DF_TRADES['Date/Time'] >= DATE].reset_index().iloc[:, 1:]

    DF_DATA = DF_DATA[DF_DATA['datetime'] <= DATE_MAX].reset_index().iloc[:, 1:]
    DF_TRADES = DF_TRADES[DF_TRADES['Date/Time'] <= DATE_MAX].reset_index().iloc[:, 1:]

    DF_DATA['Type']              = ""
    DF_DATA['Signal']            = ""
    DF_DATA['Trade #']           = ""
    DF_DATA['Price USDT']        = ""
    DF_DATA['Profit %']          = ""
    DF_DATA['Take Profit']       = ""
    DF_DATA['Status']            = ""
    DF_DATA['Key']               = ""
    DF_DATA['Success']           = ""


    for a in range(len(DF_TRADES)):

        val = DF_DATA[DF_DATA['datetime'] == (DF_TRADES.at[a, 'Date/Time'])].index[0]
        
        DF_DATA.at[val, 'Type']       = DF_TRADES.at[a, 'Type']
        DF_DATA.at[val, 'Signal']     = DF_TRADES.at[a, 'Signal']
        DF_DATA.at[val, 'Trade #']    = DF_TRADES.at[a, 'Trade #']
        DF_DATA.at[val, 'Price USDT'] = DF_DATA.at[val, 'open']
        DF_DATA.at[val, 'Profit %']   = TAKE_PROFIT

    for a in range(len(DF_DATA)):
        if DF_DATA.at[a, 'Type'] != "":
            if  "Long" in DF_DATA.at[a, 'Type']:
                DF_DATA.at[a, 'Take Profit']= ((100 + TAKE_PROFIT)/100) * DF_DATA.at[a, 'open']
            else:
                DF_DATA.at[a, 'Take Profit']= ((100 - TAKE_PROFIT)/100) * DF_DATA.at[a, 'open']

            DF_DATA.at[a, 'Status'] = "OPEN"
            DF_DATA.at[a, 'Key'] = "OPEN " + str(DF_DATA.at[a, 'Trade #'])



    return DF_TRADES, DF_DATA





In [249]:

def TRADE_PROCESSING(DF_DATA, TAKE_PROFIT):

    maxValues          = DF_DATA[DF_DATA['Trade #']!=""]
    maxValues          = maxValues['Trade #'].max()
    RANGE_LIST         = []
    RANGE_TOTAL_OUTPUT = []
    J                  = 1


    for i in range(len(DF_DATA)):

        if DF_DATA.at[i, 'Type'] != "":
            if  "Long" in DF_DATA.at[i, 'Type']:

                if DF_DATA.at[i, 'high'] >= DF_DATA.at[i, 'Take Profit']:
                    DF_DATA.at[i, 'Success'] = 'LONG OPEN - SUCCESS'
                else:
                    DF_DATA.at[i, 'Success'] = 'LONG OPEN - INCOMPLETE'

            else:
                if DF_DATA.at[i, 'low'] <= DF_DATA.at[i, 'Take Profit']:
                    DF_DATA.at[i, 'Success'] = 'SHORT OPEN - SUCCESS'
                else:
                    DF_DATA.at[i, 'Success'] = 'SHORT OPEN - INCOMPLETE'



    for i in range(1,len(DF_DATA)):

        if DF_DATA.at[i-1, 'Success'] == 'LONG OPEN - SUCCESS' or DF_DATA.at[i-1, 'Success'] == 'SHORT OPEN - SUCCESS':
                pass

        elif DF_DATA.at[i-1, 'Success'] == 'LONG OPEN - INCOMPLETE':
                DF_DATA.at[i, 'Trade #']     = DF_DATA.at[i-1, 'Trade #']
                DF_DATA.at[i, 'Price USDT']  = DF_DATA.at[i-1, 'Price USDT']
                DF_DATA.at[i, 'Take Profit'] = DF_DATA.at[i-1, 'Take Profit']
                DF_DATA.at[i, 'Profit %']    = DF_DATA.at[i-1, 'Profit %']

                if DF_DATA.at[i, 'high'] >= DF_DATA.at[i, 'Take Profit']:
                    DF_DATA.at[i, 'Success']     = 'LONG OPEN - SUCCESS'
                    DF_DATA.at[i, 'Type']        = 'Exit Long'
                    DF_DATA.at[i, 'Signal']      = 'Exit Long'
                    DF_DATA.at[i, 'Status']      = 'CLOSE'
                    DF_DATA.at[i, 'Key']         = 'CLOSE ' + str(DF_DATA.at[i, 'Trade #'])

                else:
                    DF_DATA.at[i, 'Success'] = 'LONG OPEN - INCOMPLETE'

        elif DF_DATA.at[i-1, 'Success'] == 'SHORT OPEN - INCOMPLETE':
                DF_DATA.at[i, 'Trade #']     = DF_DATA.at[i-1, 'Trade #']
                DF_DATA.at[i, 'Price USDT']  = DF_DATA.at[i-1, 'Price USDT']
                DF_DATA.at[i, 'Take Profit'] = DF_DATA.at[i-1, 'Take Profit']
                DF_DATA.at[i, 'Profit %']    = DF_DATA.at[i-1, 'Profit %']

                if DF_DATA.at[i, 'low'] <= DF_DATA.at[i, 'Take Profit']:
                    DF_DATA.at[i, 'Success']     = 'SHORT OPEN - SUCCESS'
                    DF_DATA.at[i, 'Type']        = 'Exit Short'
                    DF_DATA.at[i, 'Signal']      = 'Exit Short'
                    DF_DATA.at[i, 'Status']      = 'CLOSE'
                    DF_DATA.at[i, 'Key']         = 'CLOSE ' + str(DF_DATA.at[i, 'Trade #'])


                else:
                    DF_DATA.at[i, 'Success'] = 'SHORT OPEN - INCOMPLETE' 

        else:
                pass



    for i in range(1,len(DF_DATA)):
        if DF_DATA.at[i, 'Status'] == 'OPEN':
            DF_DATA.at[i, 'Trade #'] = J
            DF_DATA.at[i, 'Key']         = 'CLOSE ' + str(DF_DATA.at[i, 'Trade #'])

        elif DF_DATA.at[i, 'Status'] == 'CLOSE':
            DF_DATA.at[i, 'Trade #'] = J
            DF_DATA.at[i, 'Key']         = 'CLOSE ' + str(DF_DATA.at[i, 'Trade #'])
            J = J + 1

        else:
            pass



    for i in range(len(DF_DATA)):
        if DF_DATA.at[i, 'Success'] == 'LONG OPEN - SUCCESS' or DF_DATA.at[i, 'Success'] == 'SHORT OPEN - SUCCESS':
            output = TAKE_PROFIT
        else:
            output = 0

        RANGE_LIST.append(output)

    DF_DATA['OUTPUT'] = RANGE_LIST



    for i in range(len(DF_DATA)):
        if i == 0:
            VALUE_D  = DF_DATA.at[i, 'OUTPUT']
            RANGE_TOTAL_OUTPUT.append(VALUE_D)

        else:        
            DF_TEMP     = DF_DATA.iloc[:i+1 , :].copy()
            VALUE_D     = DF_TEMP['OUTPUT'].sum()
            RANGE_TOTAL_OUTPUT.append(VALUE_D)


    DF_DATA['TOTAL OUTPUT']  = RANGE_TOTAL_OUTPUT
    DF_DATA                  = DF_DATA[['datetime', 'open', 'high', 'low', 'close', 'volume', 'Type', 
                                        'Signal', 'Trade #', 'Price USDT', 'Profit %', 'Take Profit', 
                                        'Status', 'Key', 'Success', 'OUTPUT', 'TOTAL OUTPUT']].rename({'datetime' : 'DATE', 
                                                                                                            'open'     : 'OPEN',
                                                                                                            'high'     : 'HIGH',
                                                                                                            'low'      : 'LOW',
                                                                                                            'close'    : 'CLOSE',
                                                                                                            'volume'   : 'VOLUME',
                                                                                                            }, axis=1)

    DF_DATA                         = DF_DATA.set_index('DATE')


    return DF_DATA


    





In [250]:


def CONVERT_TO_BACKTEST_FORMAT(df_test, TAKE_PROFIT_USE, ROOT):

    TP                     = TAKE_PROFIT_USE
    TRADE_NBR              = 1
    df_tv                  = df_test[['OPEN', 'HIGH','LOW', 'CLOSE','Success', 'Price USDT', 'Take Profit']].rename({'Success':'OUTPUT', 'Price USDT': 'POSITION_OPEN', 'Take Profit':'TP'}, axis=1)
    
    df_tv                  = df_tv.reset_index()
    df_tv['TP_%']          = TP
    df_tv['TRADE_NBR']     = ""
    df_tv['DRAWDOWN']      = ""
    df_tv['DRAWDOWN_%']    = ""


    for i in range(len(df_tv)):
        if df_tv.at[i, 'OUTPUT'] == 'SHORT OPEN - SUCCESS' or df_tv.at[i, 'OUTPUT'] == 'LONG OPEN - SUCCESS':
                df_tv.at[i, 'TRADE_NBR']     = TRADE_NBR
                TRADE_NBR                    = TRADE_NBR + 1

        elif df_tv.at[i, 'OUTPUT'] == 'SHORT OPEN - INCOMPLETE' or df_tv.at[i, 'OUTPUT'] == 'LONG OPEN - INCOMPLETE':
                df_tv.at[i, 'TRADE_NBR']     = TRADE_NBR




    for i in range(len(df_tv)):
        if 'SHORT' in df_tv.at[i, 'OUTPUT']:
            df_tv.at[i, 'DRAWDOWN']   = df_tv.at[i, 'POSITION_OPEN'] - df_tv.at[i, 'HIGH']
            df_tv.at[i, 'DRAWDOWN_%'] = round(((df_tv.at[i, 'POSITION_OPEN'] - df_tv.at[i, 'HIGH']) / df_tv.at[i, 'POSITION_OPEN'])*100,2) 

        elif 'LONG' in df_tv.at[i, 'OUTPUT']:
            df_tv.at[i, 'DRAWDOWN']   = df_tv.at[i, 'POSITION_OPEN'] - df_tv.at[i, 'LOW']
            df_tv.at[i, 'DRAWDOWN_%'] = round(((df_tv.at[i, 'LOW'] - df_tv.at[i, 'POSITION_OPEN']) / df_tv.at[i, 'POSITION_OPEN'])*100,2)                                                    

    return df_tv

    
    
    



    

In [355]:

def FS(df_tv, ROOT, ticker):

    df_tv = df_tv[df_tv['TRADE_NBR'] != ''].reset_index()
    df_tv['DRAWDOWN_%'] = df_tv['DRAWDOWN_%'].abs()

    GROUP_DF                   = df_tv.groupby(['TRADE_NBR']).agg({'DRAWDOWN':'max', 'DRAWDOWN_%':'max'})
    GROUP_DF                   = GROUP_DF.reset_index()

    del df_tv['DRAWDOWN']
    del df_tv['DRAWDOWN_%']

    df_tv = pd.merge(df_tv, GROUP_DF, on="TRADE_NBR")
    
    df_out                     = df_tv[['TRADE_NBR', 'DATE', 'POSITION_OPEN', 'TP_%', 'DRAWDOWN_%', 'OUTPUT']].rename({'TRADE_NBR': 'Trade #', 'DATE':'Date/Time', 'POSITION_OPEN':'Price USDT', 'TP_%':'Profit %', 'DRAWDOWN_%':'Drawdown %'}, axis=1)

    df_out['Type']             = ""
    df_out['Signal']           = ""
    df_out['Contracts']        = ""
    df_out['Profit USDT']      = ""
    df_out['Cum. Profit USDT'] = ""
    df_out['Cum. Profit %']    = ""
    df_out['Run-up USDT']      = ""
    df_out['Run-up %']         = ""
    df_out['Drawdown USDT']    = ""


    df_out = df_out[['Trade #', 'Type', 'Signal', 'Date/Time', 'Price USDT', 'Contracts', 'Profit USDT', 'Profit %', 'Cum. Profit USDT', 'Cum. Profit %', 'Run-up USDT', 'Run-up %', 'Drawdown USDT', 'Drawdown %', 'OUTPUT']]
    df_out['COUNT']  = ""

    J = 1

    for i in range(0, len(df_out)):
        if i == 0 and df_out.at[i, 'Trade #'] != '':
            df_out.at[i, 'COUNT']  = J        

        elif df_out.at[i, 'Trade #'] != df_out.at[i-1, 'Trade #'] and df_out.at[i, 'Trade #'] != '':
            J = 1
            df_out.at[i, 'COUNT']  = J

        elif df_out.at[i, 'Trade #'] == df_out.at[i-1, 'Trade #'] and df_out.at[i, 'Trade #'] != '':
            J = J + 1
            df_out.at[i, 'COUNT']  = J


    GROUP_DF_2                   = df_out.groupby(['Trade #']).agg({'COUNT':'max'})
    GROUP_DF_2                   = GROUP_DF_2.reset_index()

    del df_out['COUNT']

    df_out                      = pd.merge(df_out, GROUP_DF_2, on=['Trade #'])
    df_final                    = df_out[df_out['COUNT']==1]
    df_final                    = df_final.append(df_final)
    df_temp                     = df_out[df_out['COUNT']==2]
    df_final                    = df_final.append(df_temp)
    df_temp                     = df_out[df_out['COUNT']>2].reset_index().iloc[:,1:]
    list_unique                 = df_temp["Trade #"].unique().tolist()

    print(list_unique)

    if len(list_unique) != 1:
        for k in range(len(list_unique)):
            df_temp_iter  = df_temp[df_temp['Trade #']==list_unique[k]].reset_index().iloc[:,1:]
            df_temp_first = df_temp_iter.iloc[0, :]
            df_temp_last  = df_temp_iter.iloc[len(df_temp_iter)-1, :]
            df_final      = df_final.append(df_temp_first)
            df_final      = df_final.append(df_temp_last)        

    else:
        df_temp_first = df_temp.iloc[0, :]
        df_temp_last  = df_temp.iloc[len(df_temp)-1, :]
        df_final      = df_final.append(df_temp_first)
        df_final      = df_final.append(df_temp_last)

    df_final        = df_final.sort_values(by=['Date/Time'], ascending=True).reset_index().iloc[:, 1:]
    LONG_SHORT_LIST = []

    for i in range(len(df_final)):
        if (i % 2) == 0:
            if 'SHORT' in df_final.at[i, 'OUTPUT']:
                VAL = 'ENTRY SHORT'
            elif 'LONG' in df_final.at[i, 'OUTPUT']:
                VAL = 'ENTRY LONG'            
        else:
            if 'SHORT' in df_final.at[i, 'OUTPUT']:
                VAL = 'EXIT SHORT'
            elif 'LONG' in df_final.at[i, 'OUTPUT']:
                VAL = 'EXIT LONG'             

        LONG_SHORT_LIST.append(VAL)

    df_final['Type'] = LONG_SHORT_LIST

    del df_final['OUTPUT']
    del df_final['COUNT']

    #df_final['Drawdown %'] = df_final['Drawdown %'] * -1

    df_final.to_csv(ROOT + '/' + 'MIXED STRATEGY - BUY & HOLD TP - ' + ticker + '.csv') 

    #df_final = df_final[df_final['Trade #'] == 1]



    return df_final




In [385]:
#### COMMMANDER ### 

ROOT                = (r'/Users/westhomas/Desktop/TEST FOLDER/DATA')
TAKE_PROFIT         = 0.23
DATE                = '2022-01-01'

k = 4


COINS = ['XRPUSDT', 'ETHUSDT', 'ADAUSDT', 'BTCUSDT', 'AVAXUSDT']


DF_TRADES, DF_DATA = SOURCE_DATA_MANIPULATE(TAKE_PROFIT, ROOT, DATE, k)


DF_DATA            = TRADE_PROCESSING(DF_DATA, TAKE_PROFIT)
DF_OUTPUT          = CONVERT_TO_BACKTEST_FORMAT(DF_DATA, TAKE_PROFIT, ROOT)
DF_FINAL           = FS(DF_OUTPUT, ROOT, COINS[k])


DF_FINAL.iloc[:,:]


[21, 22, 23, 24, 25, 27, 37, 39, 43, 45, 46, 51, 56, 60, 61, 64, 68, 77, 79, 88, 92]


,Trade #,Type,Signal,Date/Time,Price USDT,Contracts,Profit USDT,Profit %,Cum. Profit USDT,Cum. Profit %,Run-up USDT,Run-up %,Drawdown USDT,Drawdown %
0,1,ENTRY LONG,,2022-01-02 01:00:00,114.12,,,0.23,,,,,,0.65
1,1,EXIT LONG,,2022-01-02 01:00:00,114.12,,,0.23,,,,,,0.65
2,2,ENTRY SHORT,,2022-01-03 17:00:00,110.69,,,0.23,,,,,,0.46
3,2,EXIT SHORT,,2022-01-03 17:00:00,110.69,,,0.23,,,,,,0.46
4,3,ENTRY LONG,,2022-01-11 21:00:00,89.67,,,0.23,,,,,,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,90,EXIT LONG,,2023-03-13 01:00:00,16.17,,,0.23,,,,,,0.43
180,91,ENTRY SHORT,,2023-03-15 21:00:00,15.58,,,0.23,,,,,,0.13
181,91,EXIT SHORT,,2023-03-15 21:00:00,15.58,,,0.23,,,,,,0.13
182,92,ENTRY LONG,,2023-03-17 13:00:00,16.91,,,0.23,,,,,,3.90
